In [1]:
import sys, math, time
import re
import os
import pandas as pd
import numpy as np
from array import array

np.set_printoptions(suppress=True)

In [2]:
# PHYSICAL CONSTANTS

GAS_CONSTANT = 8.31446261815324 # J / (mol·K) #
PLANCK_CONSTANT = 6.62607015e-34 # m2 kg / s #
BOLTZMANN_CONSTANT = 1.3806488e-23 # m2·kg / (s2⋅K) or J / K # 
SPEED_OF_LIGHT = 2.99792458e10 # m / s #
AVOGADRO_CONSTANT = 6.02214076e23 # / mol #
AMU_to_KG = 1.66053886E-27 # kg #
autokcal = 627.509541
kcaltokj = 4.184
jtohartree = 2.2937104486906E+17
PI = math.pi
k = 3.166811563E-6 #Boltzmann Constant in atomic units

#new values#
protoelec = 1836.152669389233
k_real = BOLTZMANN_CONSTANT*jtohartree #approx same as below#

atom_map = {'H': 1.008,	'He': 4.0026,	'Li': 7,	'Be': 9.012183,	'B': 10.81,	'C': 12.011,	'N': 14.007,	'O': 15.999,	'F': 18.99840316,	'Ne': 20.18,	'Na': 22.9897693,	'Mg': 24.305,	'Al': 26.981538,	'Si': 28.085,	'P': 30.973762,	'S': 32.07,	'Cl': 35.45,	'Ar': 39.9,	'K': 39.0983,	'Ca': 40.08,	'Sc': 44.95591,	'Ti': 47.867,	'V': 50.9415,	'Cr': 51.996,	'Mn': 54.93804,	'Fe': 55.84,	'Co': 58.93319,	'Ni': 58.693,	'Cu': 63.55,	'Zn': 65.4,	'Ga': 69.723,	'Ge': 72.63,	'As': 74.92159,	'Se': 78.97,	'Br': 79.9,	'Kr': 83.8,	'Rb': 85.468,	'Sr': 87.62,	'Y': 88.90584,	'Zr': 91.22,	'Nb': 92.90637,	'Mo': 95.95,	'Tc': 96.90636,	'Ru': 101.1,	'Rh': 102.9055,	'Pd': 106.42,	'Ag': 107.868,	'Cd': 112.41,	'In': 114.818,	'Sn': 118.71,	'Sb': 121.76,	'Te': 127.6,	'I': 126.9045,	'Xe': 131.29,	'Cs': 132.905452,	'Ba': 137.33,	'La': 138.9055,	'Ce': 140.116,	'Pr': 140.90766,	'Nd': 144.24,	'Pm': 144.91276,	'Sm': 150.4,	'Eu': 151.964,	'Gd': 157.2,	'Tb': 158.92535,	'Dy': 162.5,	'Ho': 164.93033,	'Er': 167.26,	'Tm': 168.93422,	'Yb': 173.05,	'Lu': 174.9668,	'Hf': 178.49,	'Ta': 180.9479,	'W': 183.84,	'Re': 186.207,	'Os': 190.2,	'Ir': 192.22,	'Pt': 195.08,	'Au': 196.96657,	'Hg': 200.59,	'Tl': 204.383,	'Pb': 207,	'Bi': 208.9804,	'Po': 208.98243,	'At': 209.98715,	'Rn': 222.01758,	'Fr': 223.01973,	'Ra': 226.02541,	'Ac': 227.02775,	'Th': 232.038,	'Pa': 231.03588,	'U': 238.0289,	'Np': 237.048172,	'Pu': 244.0642,	'Am': 243.06138,	'Cm': 247.07035,	'Bk': 247.07031,	'Cf': 251.07959,	'Es': 252.083,	'Fm': 257.09511,	'Md': 258.09843,	'No': 259.101,	'Lr': 266.12,	'Rf': 267.122,	'Db': 268.126,	'Sg': 269.128,	'Bh': 270.133,	'Hs': 269.1336,	'Mt': 277.154,	'Ds': 282.166,	'Rg': 282.169,	'Cn': 286.179,	'Nh': 286.182,	'Fl': 290.192,	'Mc': 290.196,	'Lv': 293.205,	'Ts': 294.211,	'Og': 295.216}

In [41]:
#Variables to add#

NAME = 'TS-BHCl2Sub'
TEMPERATURE = 298.15

In [48]:
#Set variables to use later

DIR_NAME = os.path.join('BCl_Ionic', 'vib', NAME, NAME)
 
SM_SPC = DIR_NAME + '_IRC_B_SPC.out'
P_SPC = DIR_NAME + '_IRC_F_SPC.out'
TS_SPC = DIR_NAME + '_SPC.out'
SM = DIR_NAME + '_IRC_B_T.out'
P = DIR_NAME + '_IRC_F_T.out'
TS = DIR_NAME + '_T.out'
TS_VIB = DIR_NAME + '_T.out'
TS_XYZ = DIR_NAME + '.xyz'

In [49]:
#10-point Gauss-Legendre Quadrature abscissa and weight (exact solution for up to 21st order polynomial)
x = array('d',[-0.9739065285,-0.8650633667,-0.6794095683,-0.4333953941,-0.1488743390,0.1488743390,0.4333953941,0.6794095683,0.8650633667,0.9739065285])
w = array('d',[0.0666713443,0.1494513492,0.2190863625,0.2692667193,0.2955242247,0.2955242247,0.2692667193,0.2190863625,0.1494513492,0.0666713443])

In [50]:
#Create functions

#Read ORCA output for the Final SCF Energy
def final_scf_energy(file):
    with open(file, 'r') as ORCA_output:
        for line in ORCA_output:
            if 'FINAL SINGLE POINT ENERGY' in line:   
                spc = line.strip().split()[-1]
                return float(spc)
    return None

#Read ORCA output for the ZPE
def zero_point_energy(file):
    start_reading = False
    with open(file, 'r') as ORCA_output:
        
        for line in ORCA_output:
        # Check for the start condition
            if line.startswith('THERMOCHEMISTRY AT ' + str(TEMPERATURE) + 'K'):
                start_reading = True
                continue
            
            if start_reading:
                if 'Zero point energy' in line:   
                    ZPE = line.strip().split()[4]
                    return float(ZPE)
        return None

#Read ORCA output for thermal correction to Gibbs Free Energy
def g_corr(file):
    start_reading = False
    with open(file, 'r') as ORCA_output:
        
        for line in ORCA_output:
        # Check for the start condition
            if line.startswith('THERMOCHEMISTRY AT ' + str(TEMPERATURE) + 'K'):
                start_reading = True
                continue
            
            if start_reading:
                if 'G-E(el)' in line:   
                    gcorr = line.strip().split()[2]
                    return float(gcorr)
        return None

#Read ORCA .out for vibrational freq of normal mode of transition
def vibration(file):
    with open(file, 'r') as ORCA_output:
        for line in ORCA_output:
            if '***imaginary mode***' in line:
                novib = line.strip().split()[0].replace(':', '')
                vib = line.strip().split()[1]
                return int(novib), float(vib)
    return None

def AtomMass(file):
    with open(file) as f:
        atoms = []
        lines = f.read().splitlines()
        for i, line in enumerate(lines[2:], start=2):
            split_line = line.split()
            mass = atom_map[split_line[0]]
            atoms.append((split_line[0], mass))
    return atoms

#Calculate the force constant using calculated reduced mass similar to Gaussian
def red_mass_fc(file1,file2):
    atoms = AtomMass(file1)
    novib, vib = vibration(file2)
    novib_row = novib // 6
    masses = [item[1] for item in atoms]
    n_atoms = len(atoms)
    n_rows = n_atoms * 3
    normsq_list = []
    
    with open(file2, 'r') as ORCA_output:
        lines = ORCA_output.readlines()
        
    for i, line in enumerate(lines):
        if line.startswith('NORMAL MODES'):
            for line in lines[i + 8 + (n_rows * novib_row) + novib_row :i + 8 + (n_rows * novib_row) + novib_row + n_rows]:
                data = line.split()[1:]
                disp = float(data[novib%len(data)])
                value = disp**2
                normsq_list.append(value)
    
    red_mass_list = normsq_list.copy()
    for j in range(n_rows):
        red_mass_list[j] = normsq_list[j] / masses[j // 3]
    
    red_mass = 1 / sum(red_mass_list)
    fc = (red_mass * AMU_to_KG * 4 * PI**2 * SPEED_OF_LIGHT**2 * vib**2)/ 100
    
    return red_mass, fc

#Calculate the force constant setting the reduced mass equal to 1
def fc_redmass1(file):
    red_mass = 1.008
    vibno, vib = vibration(file)
    fc = (red_mass * AMU_to_KG * 4 * PI**2 * SPEED_OF_LIGHT**2 * float(vib)**2)/ 100 
    return red_mass, fc

In [51]:
#Parameters B, ALPHA, a, b, d of Eckart Potential
def Bee(V_max,V_r, V_p):
    bee = (V_max ** 0.5 + (V_max - (V_p - V_r)) ** 0.5) ** 2
    return bee

def ALPHA(B,F_s,V_max,V_r, V_p):
    alpha = (B * F_s / (2 * V_max * (V_max - (V_p - V_r)))) ** 0.5
    return alpha

def A(E,mu,ALPHA):
    a =  2 * PI * (2 * mu * E)**0.5 / ALPHA
    return a

def B(E,mu,V_p,V_r,ALPHA):
    b =  2 * PI * (2 * mu * (E - (V_p - V_r)))**0.5 / ALPHA
    return b

def D(bee,mu,ALPHA):
    d =  2 * PI * abs((2 * mu * bee - (ALPHA/2)**2))**0.5 / ALPHA
    return d

#Calculation of Transmission Probabilty of Eckart Potential
def T(a,b,d):
    T = (math.cosh(a+b) - math.cosh(a-b))/(math.cosh(a+b) + math.cosh(d))
    return T

#Calculation of SINH function of Kappa
def S(Vmax,E):
    S = math.sinh(((Vmax-E)) / (TEMPERATURE*k))
    return S

In [52]:
#Think about scaling#

#Input output files to extract data
V_r = float(final_scf_energy(SM_SPC)) + float(zero_point_energy(SM))
V_p = float(final_scf_energy(P_SPC)) + float(zero_point_energy(P))
V_max = (float(final_scf_energy(TS_SPC)) + float(zero_point_energy(TS)))
G_r = float(final_scf_energy(SM_SPC)) + float(g_corr(SM)) + 0.00398041762
G_p = float(final_scf_energy(P_SPC)) + float(g_corr(P)) + 0.00398041762
G_ts = float(final_scf_energy(TS_SPC)) + float(g_corr(TS)) + 0.00398041762

if V_r > V_p:
    E_o = V_r
else:
    E_o = V_p

V_max = V_max - V_r
V_p = V_p - V_r
E_o = E_o - V_r
V_r = V_r - V_r
        
y = (V_max - E_o)/2.0 
z = (V_max + E_o)/2.0 

In [53]:
# Specifing Parameters for the Eckart Potential - input .vib file

mu_uncrr, F_s_uncrr = red_mass_fc(TS_XYZ,TS_VIB)
FQn, FQ_uncrr = vibration(TS)

mu = float(mu_uncrr)*1836 #convert to electron mass#
F_s = float(F_s_uncrr)/15.569141 #convert to Hartree per bohr^2#
FQ = abs(float(FQ_uncrr))*4.55633e-6 #convert to Hartree#

bee = Bee(V_max,V_r,V_p)
alpha = ALPHA(bee,F_s,V_max,V_r,V_p)
d = D(bee,mu,alpha)

In [54]:
#Calculation of Uncorrected Gibbs Free Energy Barrier in kcal/mol and Rate
delta_g_dagg = (G_ts - G_r)*2625.5
delta_g_daggk = (G_ts - G_r)*autokcal
ln_uncorr_rate = math.log(BOLTZMANN_CONSTANT * TEMPERATURE / (PLANCK_CONSTANT))-delta_g_dagg*1000/(GAS_CONSTANT*TEMPERATURE)  
uncorr_rate = "{:.10E}".format(math.exp(ln_uncorr_rate))

#Calculation of Wigner tunneling correction
kappa_w = 1+(( FQ / (k*TEMPERATURE) )**2) / 24

#Calculation of Skodje tunneling correction
alpha_s = (2*PI/(FQ))
beta_s = (1/ (k*TEMPERATURE))

if V_p < V_r:
    Vee = 0
else:
    Vee = V_p-V_r
if alpha_s > beta_s:
    kappa_s = (beta_s*PI/alpha_s)/(math.sin(beta_s*PI/alpha_s))+(beta_s/(beta_s-alpha_s)*math.exp((beta_s-alpha_s)*(V_max-Vee)))
elif alpha_s < beta_s:
    kappa_s = (beta_s/(beta_s-alpha_s)*(math.exp((beta_s-alpha_s)*(V_max-Vee))-1))
else:
    kappa_s = alpha_s*(V_max-Vee)
       
#Calculation of Eckart tunneling correction using 10-point Gauss-Legendre Quadrature
kappa = 1
for i in range(0,10):
    a = A((x[i] * y + z),mu,alpha)
    b = B((x[i] * y + z),mu,V_p,V_r,alpha)
    kappa = (2 * y  / (TEMPERATURE*k) * w[i] * S((V_max),(x[i] * y + z)) * T(a,b,d)) + kappa

#Calculation of Wigner Apparent Gibbs Free Energy Barrier in kcal/mol and Rate
corr_rate_w = "{:.10E}".format(kappa_w*math.exp(ln_uncorr_rate))

delta_g_dagg_app_w = GAS_CONSTANT * TEMPERATURE * (math.log(BOLTZMANN_CONSTANT * TEMPERATURE / (PLANCK_CONSTANT)) - math.log(kappa_w) - ln_uncorr_rate) /1000
delta_g_dagg_app_wk = delta_g_dagg_app_w / kcaltokj
w_cor = delta_g_dagg_app_wk - delta_g_daggk  

#Calculation of Skodje Apparent Gibbs Free Energy Barrier in kcal/mol and Rate
corr_rate_s = "{:.10E}".format(kappa_s*math.exp(ln_uncorr_rate))

delta_g_dagg_app_s = GAS_CONSTANT * TEMPERATURE * (math.log(BOLTZMANN_CONSTANT * TEMPERATURE / (PLANCK_CONSTANT)) - math.log(kappa_s) - ln_uncorr_rate) /1000
delta_g_dagg_app_sk = delta_g_dagg_app_s / kcaltokj
s_cor = delta_g_dagg_app_sk - delta_g_daggk

#Calculation of Eckart Apparent Gibbs Free Energy Barrier in kJ/mol and Rate
corr_rate = "{:.10E}".format(kappa*math.exp(ln_uncorr_rate))

delta_g_dagg_app = GAS_CONSTANT * TEMPERATURE * (math.log(BOLTZMANN_CONSTANT * TEMPERATURE / (PLANCK_CONSTANT)) - math.log(kappa) - ln_uncorr_rate) /1000
delta_g_dagg_appk = delta_g_dagg_app / kcaltokj
e_cor = delta_g_dagg_appk - delta_g_daggk

In [55]:
#Print section

print(FQ_uncrr)

data = []
data.append([NAME, kappa_w, kappa_s, kappa, uncorr_rate, corr_rate_w, corr_rate_s, corr_rate, delta_g_daggk,delta_g_dagg_app_wk,delta_g_dagg_app_sk,delta_g_dagg_appk])

df = pd.DataFrame(data, columns=['Name', 'Kw', 'Ksw', 'Ke','k','kw','ksw','ke','G*','G*w','G*sw','G*e'])

pd.set_option('display.float_format', '{:.2f}'.format)

df

-141.02


,Name,Kw,Ksw,Ke,k,kw,ksw,ke,G*,G*w,G*sw,G*e
0,TS-BHCl2Sub,1.02,0.94,1.00,4.8886110980E+12,4.9829415551E+12,4.6187914884E+12,4.8932556007E+12,0.14,0.13,0.18,0.14


In [56]:
#Rate calculator

activation = (16 / autokcal) * 2625.5

ln_act_rate = math.log(BOLTZMANN_CONSTANT * TEMPERATURE / (PLANCK_CONSTANT))-activation*1000/(GAS_CONSTANT*TEMPERATURE)  
rate = "{:.10E}".format(math.exp(ln_act_rate))
print(rate)

1.1619307847E+01


In [98]:
adjrate = "{:.10E}".format(math.exp(ln_act_rate)*2)
print(adjrate)

2.3238615693E+01


In [70]:
kappa_w

1.023353533638133

In [71]:
kappa_s

-0.4372748723052966

In [72]:
FQ_uncrr

-155.14